### RF Model

### cross evaluate the performance of each predictor individually based on its R2

#### preprocessing for Random Forest

0. load and sort data frame (temporal dependency is overcome)

1. remove instances with imputed target variables, since temporal consistency is not needed for random forest & missing data type is completely at random --> no introduced bias

2. add column of weighted_mean_pollution: for each timestamp (distance based) mean of pollution from other stations 

3. separate 10 percent of instances of each group (id) for test propose, rest for training and validation

4. incremental feature section based on prior MI and VIF based feature selection

5. hyperparamter tuning

In [7]:
# 0. load (and sort) data frame
seed = 42
main = pd.read_csv('data/df_processed.csv').sample(frac=1, random_state= seed)
main.columns

Index(['Unnamed: 0', 'MESS_DATUM', 'id', 'Stickstoffdioxid', 'prec_mm',
       'prec_bool', 'humidity', 'temp', 'radiation', 'wind_degree',
       'wind_speed', 'air_pressure', 'free_wind', 'prop_intercept_200',
       'prop_intercept_50', 'GVI_25', 'GVI_50', 'GVI_75', 'GVI_100', 'GVI_200',
       'tvi_25', 'tvi_50', 'tvi_75', 'tvi_100', 'tvi_200', 'prop_main_',
       'nearest_st', 'nearest_in', 'pop_200', 'pop_500', 'weekend', 'rushhour',
       'lai_factor', 'distance_city'],
      dtype='object')

In [8]:
# 1. remove instances with imputed target variable
import pickle

# load indexes of imputed values
with open('data/timesteps_missing_data.pkl', 'rb') as pickle_file:
    timestep_missing_data = pickle.load(pickle_file)


# exclude values 
main_no_impute = main.copy()

for id in timestep_missing_data.keys():  
    for timestep in timestep_missing_data.get(id):
        #exclude row which match indexing (id and timestep of imputed value)
        main_no_impute = main_no_impute[~((main_no_impute['id'] == id) & (main_no_impute['time_step'] == timestep))]

print(f'Imputed Instances excluded: {main.shape[0] - main_no_impute.shape[0]}')

Imputed Instances excluded: 722


In [9]:
# 2. add column of weighted mean pollution
import numpy as np

# Function to calculate weighted mean pollution based on similarity in distance to the city center
def weighted_mean_pollution(row):

    # Filter for other stations at the same timestamp
    same_time = main[main['time_step'] == row['time_step']]
    
    # Exclude the current station
    other_stations = same_time[same_time['id'] != row['id']]
    
    # similarity weights based on the inverse of the absolute difference in distances
    weights = 1 / (1 + np.abs(other_stations['distance_city'] - row['distance_city']))
       
    # Calculate the weighted mean pollution
    weighted_mean = np.average(other_stations['NO2'], weights=weights)

    return weighted_mean


# add weighted mean pollution for each time step
main_no_impute['weighted_mean_pollution'] = main_no_impute.apply(weighted_mean_pollution, axis=1)    

In [10]:
# 3. separate test from training station based on id 
# TEST STATIONS: mc117, mc018, mc145
main = main_no_impute.copy()
test_data_mc117 = main[main['id'] == 'mc117'].sort_values('time_step')
test_data_mc018 = main[main['id'] == 'mc018'].sort_values('time_step')
test_data_mc145 = main[main['id'] == 'mc145'].sort_values('time_step')

train_df = main[~main['id'].isin(['mc117', 'mc018', 'mc145'])]
train_df['weighted_mean_pollution'] = train_df.apply(weighted_mean_pollution, axis=1)    

C:\Users\richa\AppData\Local\Temp\ipykernel_21448\73997058.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['weighted_mean_pollution'] = train_df.apply(weighted_mean_pollution, axis=1)


In [11]:
# 2.1 create 18 look up tables for weighted pollution 
def construct_LOOCV_weighted_pollution(df):
    for site in list(df['id'].unique()):
        current_df = df[df['id'] != site][['time_step', 'id', 'NO2','distance_city']]
        current_df['weighted_pollution_df'] = current_df.apply(weighted_mean_pollution, axis=1)
        current_df = current_df[['time_step', 'id', 'weighted_pollution_df']]
        current_df.to_csv(f'data/pollution_data/LOOCV/weighted_pollution_no_{site}.csv')

construct_LOOCV_weighted_pollution(df = train_df)

#### calculate feature importance through model diagnostics with all features

In [12]:
# simple cross- validation for  feature importance test
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


def site_wise_RF(df):

    # store r2 and mae performance
    performance = {}

    # store feature wise performance
    index_names = list(df.columns.drop(['Unnamed: 0', 'time_step', 'id', 'NO2']))
    feature_performance = pd.DataFrame(index=index_names)
    main = df.copy()

    n = 0 
    # initiate iteration over sites


    for site in list(main['id'].unique()):

        # initialize test set
        X_test = main[main['id'] == site].drop(['Unnamed: 0', 'time_step', 'id', 'NO2','distance_city'], axis = 1)
        y_test = main[main['id'] == site]['NO2']

        # initialize training set + weighted pollution 
        train = main[main['id'] != site].drop('weighted_mean_pollution', axis = 1)
        
        train['weighted_mean_pollution'] = train.apply(weighted_mean_pollution, axis=1)    

        X_train = train.drop(['Unnamed: 0', 'time_step', 'id', 'NO2','distance_city'], axis = 1)
        y_train = train['NO2']

        # train model 
        model = RandomForestRegressor(n_jobs=-1) # initiate model
        model.fit(X_train, y_train) # train model based on single feature
           
        # evaluate model on r2 and mse    
        y_pred = model.predict(X_test) # evaluate model
        r2  = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred, squared=False)

        performance[site] = [r2, mse] # store performance
                
        # feature importance
        global_importances = pd.Series(100*model.feature_importances_, index=X_train.columns)
        feature_performance = pd.concat([feature_performance, global_importances], axis=1)
        
        # return process 
        n += 1
        if n%4 == 0:
            print(f"Process: {100 * (n / len(list(main['id'].unique())))}")

    print(X_train.columns)

    return feature_performance, performance

features = ['Unnamed: 0', 'time_step', 'id', 'NO2', 'free_wind', 'prop_intercept_200',
'prop_intercept_50', 'GVI_25', 'GVI_50', 'GVI_75', 'GVI_100', 'GVI_200',
'tvi_25', 'tvi_50', 'tvi_75', 'tvi_100', 'tvi_200', 'prop_main_',
'nearest_st', 'nearest_in', 'pop_200', 'pop_500', 'weekend', 'rushhour',
'lai_factor', 'distance_city', 'weighted_mean_pollution']

#feature_performance, performance = site_wise_RF(df = train_df[ features])
feature_performance, performance = site_wise_RF(df = train_df)

feature_performance_ordered = list(feature_performance.index)

# per site performance
performance_pf = pd.DataFrame(performance)
performance_pf['total'] = performance_pf.mean(axis=1)
performance_pf = performance_pf.transpose().rename({0: 'R2', 1 : 'MAE' }, axis = 1)
#performance_pf.to_csv('data/output/baseline_performance/R2_MAE_2.csv')
performance_pf

Process: 30.76923076923077
Process: 61.53846153846154
Process: 92.3076923076923
Index(['prec_mm', 'prec_bool', 'humidity', 'temp', 'radiation', 'wind_degree',
       'wind_speed', 'air_pressure', 'free_wind', 'prop_intercept_200',
       'prop_intercept_50', 'GVI_25', 'GVI_50', 'GVI_75', 'GVI_100', 'GVI_200',
       'tvi_25', 'tvi_50', 'tvi_75', 'tvi_100', 'tvi_200', 'prop_main_',
       'nearest_st', 'nearest_in', 'pop_200', 'pop_500', 'weekend', 'rushhour',
       'lai_factor', 'weighted_mean_pollution'],
      dtype='object')


,R2,MAE
mc221,0.743825,6.812780
mc115,0.077539,10.560281
mc282,0.636920,5.496195
mc190,0.748814,7.333007
mc042,0.824352,4.581197
mc174,0.604628,8.326037
mc077,0.531797,4.668812
mc010,0.716982,5.906101
mc027,0.545918,5.069790
mc032,0.489451,4.908707


In [13]:
# simple cross- validation for  feature importance test
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


def site_wise_RF2(df, features,parameter = None):

    # store r2 and mae performance
    performance = {}

    main = df.copy()

    n = 0 
    # initiate iteration over sites


    for site in list(main['id'].unique()):

        # initialize test set
        X_test = main[main['id'] == site][features] .drop(['NO2'], axis = 1)
        y_test = main[main['id'] == site]['NO2']

        # initialize training set + weighted pollution 
        train = main[main['id'] != site].drop('weighted_mean_pollution', axis = 1)
        train_weighted_pollution = pd.read_csv(f'data/pollution_data/LOOCV/weighted_pollution_no_{site}.csv')
        train_weighted_pollution = train_weighted_pollution.rename({'weighted_pollution_df':'weighted_mean_pollution'}, axis=1)
        train = pd.concat([train.reset_index(), train_weighted_pollution.reset_index()], axis= 1)
        
        X_train = train[features].drop(['NO2'], axis = 1)
        y_train = train['NO2']

        # train model 
        if parameter == None:
            model = RandomForestRegressor(n_jobs=-1) # initiate model
        else:
            model = RandomForestRegressor(**parameter, n_jobs=-1) # initiate model with defined parameter
        model.fit(X_train, y_train) # train model based on single feature
           
        # evaluate model on r2 and mse    
        y_pred = model.predict(X_test) # evaluate model
        performance[site] = [r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred, squared=False)] # store performance
         
        
        # return process 
        n += 1
        if n%4 == 0:
            print(f"Process: {100 * (n / len(list(main['id'].unique())))}")

    print(X_train.columns)

    return performance

features = ['Unnamed: 0', 'time_step', 'id', 'NO2', 'free_wind', 'prop_intercept_200',
'prop_intercept_50', 'GVI_25', 'GVI_50', 'GVI_75', 'GVI_100', 'GVI_200',
'tvi_25', 'tvi_50', 'tvi_75', 'tvi_100', 'tvi_200', 'prop_main_',
'nearest_st', 'nearest_in', 'pop_200', 'pop_500', 'weekend', 'rushhour',
'lai_factor', 'distance_city', 'weighted_mean_pollution']

#feature_performance, performance = site_wise_RF(df = train_df[ features])


In [14]:
# incremental feature selection
def return_mean_r2(performance, metrics = 'R2'):
    performance_pf = pd.DataFrame(performance)
    performance_pf['total'] = performance_pf.mean(axis=1)
    performance_pf = performance_pf.transpose().rename({0: 'R2', 1 : 'MAE' }, axis = 1)
    return performance_pf.loc['total', metrics]
    

def feature_section(feature_performance_ordered):

    current_features = feature_performance_ordered[1:15] # initialize start with 5 best performing features
    remaining_features = feature_performance_ordered[15:] # define remaining features
    
    # iterate over ordered list of remaining features
    for feature in remaining_features:
        current_features.append(feature)

        performance = site_wise_RF2(df = train_df, features= ['NO2'] + current_features + ['weighted_mean_pollution'])
        
        mean_performance = return_mean_r2(performance)
                
        print(f"Performance R2: {mean_performance}, mse: {return_mean_r2(performance, metrics= 'MAE')} with {len(current_features)} features.")

    return performance, current_features

#feature_performance_ordered = list(pd.read_csv('data/output/baseline_performance/feature_importance_1.csv')['Unnamed: 0'])
feature_performance_ordered = ['weighted_mean_pollution','nearest_in', 'prop_intercept_200', 'pop_500','GVI_25',
            'prop_main_','tvi_200', 'free_wind','lai_factor','wind_speed','temp','wind_degree',
            'rushhour', 'weekend', 'prec_mm', 'prec_bool']


performance, current_features = feature_section(feature_performance_ordered)


Process: 30.76923076923077
Process: 61.53846153846154
Process: 92.3076923076923
Index(['nearest_in', 'prop_intercept_200', 'pop_500', 'GVI_25', 'prop_main_',
       'tvi_200', 'free_wind', 'lai_factor', 'wind_speed', 'temp',
       'wind_degree', 'rushhour', 'weekend', 'prec_mm', 'prec_bool',
       'weighted_mean_pollution'],
      dtype='object')
Performance R2: -0.8838482253569419, mse: 12.838350031962971 with 15 features.


'tvi_25', 'tvi_50', 'wind_degree', 'temp', 'humidity', 'wind_speed',
'air_pressure', 'lai_factor', 'nearest_st', 'radiation', 'rushhour',
'pop_500', 'prop_main_', 'pop_200', 'prop_intercept_200', 'tvi_75',
'GVI_200', 'distance_city', 'prop_intercept_50', 'nearest_in',
'tvi_200', 'weighted_mean_pollution'


Performance R2: 0.5864718786581627, mse: 7.011109526152897 with 21 features.

### Random Forest Hyperparameter Testing  

1. run iteratively over all hyperparameter
2. remain best hyperparameter

In [15]:
def hyperparameter_search_random_forest(df, features, param_grid, best_performance):
    '''
    Iterate over parameter grid and rerun RF with parameters and store best performing per parameter
    '''
    best_performance = best_performance # adjust to previous best performance
    best_parameter = {}
    current_parameter = {}

    for hyperparameter in param_grid.keys():
        current_parameter = best_parameter
        for parameter in list(param_grid.get(hyperparameter)):
            current_parameter[hyperparameter] = parameter
            
            performance = site_wise_RF2(df = df,
                                        features= features,
                                        parameter = current_parameter)
            
            mean_performance = return_mean_r2(performance)        
            
            if mean_performance > best_performance:
                best_performance = mean_performance
                best_parameter[hyperparameter] = parameter
                print(f'new best parameter: {best_parameter} with R2: {mean_performance}')
            # run function with parameter & hyperparamter
            print(f'R2 of: {mean_performance} with {hyperparameter} and {current_parameter}')


# define parameter grid to search over
param_grid = {'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
              'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
              'max_features': [1.0, 'sqrt'],
              'min_samples_leaf': [1, 2, 4],
              'min_samples_split': [2, 5, 10],
              'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000] }

param_grid = {'max_depth': [None], 'max_features': ['sqrt'], 'min_samples_leaf': [4],
              'min_samples_split': [5], 'criterion': ['squared_error', 'friedman_mse', 'poisson']}

# define used features
features= ['NO2', 'tvi_25', 'wind_degree', 'temp', 'humidity', 'wind_speed',
       'air_pressure', 'lai_factor', 'pop_500', 'prop_main_',
       'prop_intercept_200', 'GVI_200', 'prop_intercept_50', 'tvi_200',
       'free_wind', 'tvi_50', 'nearest_st', 'weighted_mean_pollution']
      
hyperparameter_search_random_forest(df = train_df, features = features, param_grid = param_grid, best_performance = 0.5)


Process: 30.76923076923077
Process: 61.53846153846154
Process: 92.3076923076923
Index(['tvi_25', 'wind_degree', 'temp', 'humidity', 'wind_speed',
       'air_pressure', 'lai_factor', 'pop_500', 'prop_main_',
       'prop_intercept_200', 'GVI_200', 'prop_intercept_50', 'tvi_200',
       'free_wind', 'tvi_50', 'nearest_st', 'weighted_mean_pollution'],
      dtype='object')
new best parameter: {'max_depth': None} with R2: 0.6057801203855591
R2 of: 0.6057801203855591 with max_depth and {'max_depth': None}
Process: 30.76923076923077
Process: 61.53846153846154
Process: 92.3076923076923
Index(['tvi_25', 'wind_degree', 'temp', 'humidity', 'wind_speed',
       'air_pressure', 'lai_factor', 'pop_500', 'prop_main_',
       'prop_intercept_200', 'GVI_200', 'prop_intercept_50', 'tvi_200',
       'free_wind', 'tvi_50', 'nearest_st', 'weighted_mean_pollution'],
      dtype='object')
new best parameter: {'max_depth': None, 'max_features': 'sqrt'} with R2: 0.6119440989804145
R2 of: 0.6119440989804145 w